# Elastyczny Notatnik FLUX z Interfejsem Gradio (Wersja Finalna)

Ten notatnik przygotowuje środowisko, pobiera wszystkie 5 LoR wymaganych do odtworzenia obrazu "Hippie Woman 1966" z Civitai, a następnie uruchamia zaawansowany interfejs Gradio, który pozwala na pełną kontrolę nad każdą z LoR. Domyślne wartości są ustawione tak, aby od razu replikować docelowy obraz z poprawnymi siłami LoR.

## KROK 0: Konfiguracja Środowiska Wykonawczego

Upewnij się, że używasz akceleratora **T4 GPU** (Runtime -> Change runtime type).

## KROK 1: Instalacja Zależności i Automatyczny Restart

**INSTRUKCJA:**
1.  Uruchom poniższą komórkę.
2.  **Poczekaj**, aż zakończy swoje działanie. Sesja zostanie automatycznie przerwana (to jest normalne i oczekiwane!).
3.  Po tym, jak sesja się zrestartuje, **ręcznie uruchom komórkę z KROKU 2**.

In [ ]:
# KOMÓRKA 1: INSTALACJA I AUTOMATYCZNY RESTART
!pip install "numpy<2"
!pip install torch==2.1.0 torchvision==0.16.0 xformers==0.0.22.post7 --index-url https://download.pytorch.org/whl/cu118
!pip install -q torchsde einops diffusers accelerate gradio==3.50.2 python-multipart==0.0.12
print("\n✅ Instalacja zakończona. Inicjuję automatyczny restart kernela...")
import os
os.kill(os.getpid(), 9)

## KROK 2: Uruchomienie Aplikacji (Po Restarcie!)

Tę komórkę uruchom **DOPIERO PO** restarcie sesji. Po uruchomieniu, otwórz publiczny link do interfejsu Gradio, który pojawi się na dole.

In [ ]:
# KOMÓRKA 2: URUCHOMIENIE (WERSJA Z PEŁNĄ KONTROLĄ W GRADIO)

%cd /content
!git clone -b totoro4 https://github.com/camenduru/ComfyUI /content/TotoroUI
%cd /content/TotoroUI
!apt -y install -qq aria2

print("Pobieranie podstawowych modeli...")
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-dev-fp8.safetensors -d /content/TotoroUI/models/unet -o flux1-dev-fp8.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/ae.sft -d /content/TotoroUI/models/vae -o ae.sft
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/clip_l.safetensors -d /content/TotoroUI/models/clip -o clip_l.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/t5xxl_fp8_e4m3fn.safetensors -d /content/TotoroUI/models/clip -o t5xxl_fp8_e4m3fn.safetensors

print("Pobieranie wymaganych modeli LoRA...")
lora_files = {
    "GuyArochPhotography": "https://civitai.com/api/download/models/427902",
    "PhoneQuality": "https://civitai.com/api/download/models/131881",
    "EsteticHighHeels": "https://civitai.com/api/download/models/132839",
    "FluxlissimoAURA": "https://civitai.com/api/download/models/428235",
    "FluxlissimoCinematic": "https://civitai.com/api/download/models/428236"
}
for name, url in lora_files.items():
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}" -d /content/TotoroUI/models/loras -o {name}.safetensors
lora_names = [name + ".safetensors" for name in lora_files.keys()] + ["None"]

# Importy i przygotowanie środowiska
import random, torch, numpy as np, sys
from PIL import Image
sys.path.append('/content/TotoroUI')
import nodes, gradio as gr
from nodes import NODE_CLASS_MAPPINGS
from totoro_extras import nodes_custom_sampler
from totoro import model_management

# Przygotowanie klas do ładowania
LoraLoader = NODE_CLASS_MAPPINGS["LoraLoader"]()
RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()
UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
DualCLIPLoader = NODE_CLASS_MAPPINGS["DualCLIPLoader"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()

print("Ładowanie modeli podstawowych...")
with torch.inference_mode():
    unet, clip, vae = UNETLoader.load_unet("flux1-dev-fp8.safetensors", "fp8_e4m3fn")[0], DualCLIPLoader.load_clip("t5xxl_fp8_e4m3fn.safetensors", "clip_l.safetensors", "flux")[0], VAELoader.load_vae("ae.sft")[0]
print("Modele załadowane pomyślnie!")

def closestNumber(n, m):
    q = int(n / m); n1 = m * q
    if (n * m) > 0: n2 = m * (q + 1)
    else: n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2): return n1
    return n2

@torch.inference_mode()
def generate(positive_prompt, width, height, seed, steps, sampler_name, scheduler, guidance, *lora_params):
    global unet, clip
    if seed == 0: seed = random.randint(0, 18446744073709551615)
    print(f"Użyte ziarno (seed): {seed}")
    
    # Ładowanie LoR w pętli
    temp_unet, temp_clip = unet, clip
    for i in range(0, len(lora_params), 2):
        lora_name = lora_params[i]
        lora_strength = lora_params[i+1]
        if lora_name != "None" and lora_strength > 0:
            print(f"Ładowanie LoRA: {lora_name} z siłą {lora_strength}")
            temp_unet, temp_clip = LoraLoader.load_lora(temp_unet, temp_clip, lora_name, lora_strength, lora_strength)
    unet_lora, clip_lora = temp_unet, temp_clip
    
    cond, pooled = clip_lora.encode_from_tokens(clip_lora.tokenize(positive_prompt), return_pooled=True)
    cond = [[cond, {"pooled_output": pooled}]]
    noise = RandomNoise.get_noise(seed)[0]
    guider = BasicGuider.get_guider(unet_lora, cond, guidance)[0]
    sampler = KSamplerSelect.get_sampler(sampler_name)[0]
    sigmas = BasicScheduler.get_sigmas(unet_lora, scheduler, steps, 1.0)[0]
    latent_image = EmptyLatentImage.generate(closestNumber(width, 16), closestNumber(height, 16))[0]
    sample, sample_denoised = SamplerCustomAdvanced.sample(noise, guider, sampler, sigmas, latent_image)
    decoded = VAEDecode.decode(vae, sample)[0].detach()
    Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save("/content/flux.png")
    return "/content/flux.png"

with gr.Blocks(analytics_enabled=False) as demo:
    with gr.Row():
        with gr.Column(scale=2):
            positive_prompt = gr.Textbox(lines=5, interactive=True, value="A captivating, photorealistic image with RAW quality, cinematic grain, and eye-catching light. The scene features a 26-year-old hippie woman from 1966, radiating seductive, retro charm. She wears vibrant, psychedelic attire—a boldly patterned blouse with bell sleeves and a matching mini-skirt. Her long hair is styled in a classic '60s updo, accented with a headband, completing her look. Posing confidently in front of an orange american muscle car of 1960s, surrounded by industrial buildings on the seaport. her smile and gaze exude an irresistible allure. This image perfectly captures the essence of 1960s counterculture, blending nostalgia with timeless beauty. The shoot features vibrant colors, hyper-detailed dynamic style, with a glamorous and boudoir flair", label="Prompt")
            with gr.Accordion("LoRA 1-2", open=True):
                lora_name_1 = gr.Dropdown(lora_names, label="LoRA 1 Name", value="GuyArochPhotography.safetensors")
                lora_strength_1 = gr.Slider(minimum=0, maximum=2, value=0.7, step=0.05, label="LoRA 1 Strength")
                lora_name_2 = gr.Dropdown(lora_names, label="LoRA 2 Name", value="PhoneQuality.safetensors")
                lora_strength_2 = gr.Slider(minimum=0, maximum=2, value=0.25, step=0.05, label="LoRA 2 Strength")
            with gr.Accordion("LoRA 3-5", open=True):
                lora_name_3 = gr.Dropdown(lora_names, label="LoRA 3 Name", value="EsteticHighHeels.safetensors")
                lora_strength_3 = gr.Slider(minimum=0, maximum=2, value=1.0, step=0.05, label="LoRA 3 Strength")
                lora_name_4 = gr.Dropdown(lora_names, label="LoRA 4 Name", value="FluxlissimoAURA.safetensors")
                lora_strength_4 = gr.Slider(minimum=0, maximum=2, value=1.5, step=0.05, label="LoRA 4 Strength")
                lora_name_5 = gr.Dropdown(lora_names, label="LoRA 5 Name", value="FluxlissimoCinematic.safetensors")
                lora_strength_5 = gr.Slider(minimum=0, maximum=2, value=1.2, step=0.05, label="LoRA 5 Strength")
            generate_button = gr.Button("Generate", variant="primary")
        with gr.Column(scale=1):
            output_image = gr.Image(label="Generated image", interactive=False)
            with gr.Accordion("Główne Parametry", open=True):
                width = gr.Slider(minimum=256, maximum=2048, value=1024, step=16, label="width")
                height = gr.Slider(minimum=256, maximum=2048, value=1024, step=16, label="height")
                seed = gr.Slider(minimum=0, maximum=18446744073709551615, value=425215417, step=1, label="seed (0=random)")
            with gr.Accordion("Parametry Samplera", open=True):
                steps = gr.Slider(minimum=4, maximum=50, value=30, step=1, label="steps")
                guidance = gr.Slider(minimum=0, maximum=20, value=5.0, step=0.5, label="CFG Scale / Guidance")
                sampler_name = gr.Dropdown(["euler", "heun", "heunpp2", "dpm_2", "lms", "dpmpp_2m", "ipndm", "deis", "ddim", "uni_pc", "uni_pc_bh2"], label="sampler_name", value="dpmpp_2m")
                scheduler = gr.Dropdown(["normal", "sgm_uniform", "simple", "ddim_uniform"], label="scheduler", value="simple")

    all_inputs = [positive_prompt, width, height, seed, steps, sampler_name, scheduler, guidance, 
                  lora_name_1, lora_strength_1, lora_name_2, lora_strength_2, lora_name_3, lora_strength_3, 
                  lora_name_4, lora_strength_4, lora_name_5, lora_strength_5]
    generate_button.click(fn=generate, inputs=all_inputs, outputs=output_image)

print("\nUruchamiam interfejs Gradio...")
demo.queue().launch(inline=False, share=True, debug=True)